In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tsmoothie.smoother import GaussianSmoother
import spikeinterface
import spikeinterface.full as si
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as sp
import spikeinterface.preprocessing as spre
import spikeinterface.qualitymetrics as qm




In [2]:
#Reading the file, BP filtering
local_path= '/home/mmpatil/Documents/spikesorting/Data/231503/Activity/data.raw.h5' #network data from chip 16848

recording = se.read_maxwell(local_path,rec_name='rec0001')



inside get_reader
"Unable to open object (object 'routed' doesn't exist)"
Inside NeoBaseExtractor


AttributeError: 'MaxwellRecordingExtractor' object has no attribute 'neo_reader'

In [ ]:
channel_ids = recording.get_channel_ids()
fs = recording.get_sampling_frequency()
num_chan = recording.get_num_channels()
num_seg = recording.get_num_segments()
total_recording = recording.get_total_duration()

#print('Channel ids:', channel_ids)
print('Sampling frequency:', fs)
print('Number of channels:', num_chan)
print('Number of segments:', num_seg)
print(f"total_recording: {total_recording} s")

recording_bp = spre.bandpass_filter(recording, freq_min=300, freq_max=6000)

recording_cmr = spre.common_reference(recording_bp, reference='global', operator='median')

#recording_whtiened = spre.whiten(recording_cmr)
recording_chunk = recording_cmr.frame_slice(start_frame= 1*fs,end_frame=599*fs)

In [ ]:
default_KS3_params = ss.get_default_sorter_params('kilosort3')
print(default_KS3_params)

In [ ]:

default_KS3_params['keep_good_only'] = True
default_KS3_params['detect_threshold'] = 24
default_KS3_params['projection_threshold']=[30, 30]
default_KS3_params['preclust_threshold'] = 26
default_KS3_params['n_jobs'] = 64
print(default_KS3_params)
run_sorter = ss.run_kilosort3(recording_chunk, output_folder="kilosort3_network_firingrate", docker_image= "kilosort3-maxwellcomplib:latest",verbose=True, **default_KS3_params)

In [ ]:
sorting_KS3 = ss.Kilosort3Sorter._get_result_from_folder('/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/kilosort3_network_firingrate/sorter_output/')
total_units = sorting_KS3.get_unit_ids()
print(total_units)
print(len(total_units))
channel_ids = recording_chunk.get_channel_ids()
print(channel_ids)
channel_association_dict = {int(y):x for x,y in enumerate(channel_ids) }
print(channel_association_dict)

In [ ]:
job_kwargs = dict(n_jobs=64, chunk_duration="1s", progress_bar=True)
#waveforms = si.extract_waveforms(recording_chunk2,sorting_KS3_test,folder="./waveforms",overwrite=True, ms_before=1., ms_after=2.)
waveforms = si.extract_waveforms(recording_chunk,sorting_KS3,folder='./waveforms',ms_before=3., ms_after=4.,
    max_spikes_per_unit=500,load_if_exists=True,**job_kwargs)
print(waveforms)

In [ ]:
import spikeinterface.qualitymetrics as qm
job_kwargs = dict(n_jobs=64, progress_bar=True)
metrics = qm.compute_quality_metrics(waveforms,**job_kwargs)


In [ ]:
display(metrics)

In [ ]:
pc = sp.compute_principal_components(waveforms, n_components = 3,**job_kwargs)

In [ ]:
print(pc)

In [ ]:
w = sw.plot_unit_summary(waveforms, unit_id= 1)

In [ ]:
template_metrics = sp.calculate_template_metrics(waveforms)
display(template_metrics)

In [ ]:

isi_violations_ratio, isi_violations_count = qm.compute_isi_violations(waveforms, isi_threshold_ms=1.0)
print(isi_violations_ratio)
print(isi_violations_count)

rp_contamination,rp_violation = qm.compute_refrac_period_violations(waveforms)
print(rp_contamination)
print(rp_violation)

In [ ]:
print(waveforms.recording)

In [ ]:
random_chunk_kwargs_dict = {'seed': 111}
snr_ratio = qm.compute_snrs(waveforms,random_chunk_kwargs_dict= random_chunk_kwargs_dict)
print(snr_ratio)

In [ ]:
firing_rate = qm.compute_firing_rates(waveforms)
print(firing_rate)

In [ ]:
extremum_channels_ids =spikeinterface.full.get_template_extremum_channel(waveforms, peak_sign='neg')
print(extremum_channels_ids)

In [ ]:
violated_units = [unit for unit, ratio in isi_violations_ratio.items() if ratio > 0.0]
print(violated_units)
print(f"isi violated units:{len(violated_units)}")

refrct_violated_units = [unit for unit,ratio in rp_contamination.items() if ratio >0.0]
print(refrct_violated_units)
print(f"refract vio units:{len(refrct_violated_units)}")

In [ ]:
waveforms_copy = waveforms
import spikeinterface.curation 
sorter_red= spikeinterface.curation.remove_redundant_units(waveforms_copy)

print(sorter_red)

In [ ]:
print(sorting_KS3)

clean_sorting = sorting_KS3.remove_units(refrct_violated_units)
print(clean_sorting)
good_units = [units for units in total_units if units not in refrct_violated_units ]
print(good_units)

#now getting the wavefrom extractor

waveform_good = waveforms.select_units(good_units,new_folder='waveforms_good2')
print(waveform_good)

In [ ]:
unit_ids = clean_sorting.get_unit_ids()

print(unit_ids)

In [ ]:
import spikeinterface.postprocessing as sp

locations = sp.compute_unit_locations(waveform_good)
print(locations)

%matplotlib widget
ax = plt.subplot(111)
sw.plot_probe_map(recording,ax=ax,with_channel_ids=True)
for x,y in locations:
    ax.scatter(x,y)

In [ ]:
extremum_channels_ids =spikeinterface.full.get_template_extremum_channel(waveform_good, peak_sign='neg')
print(extremum_channels_ids)

# unique_templates_electrodes = {template: electrode 
#                                for template,electrode in extremum_channels_ids.items() 
#                                if list(extremum_channels_ids.values()).count(electrode)==1}
# print(unique_templates_electrodes)
# print(len(unique_templates_electrodes))

In [ ]:
def get_key_by_value(d, value):
    keys = [k for k, v in d.items() if v == value]
    return keys[0] if keys else None

print(get_key_by_value(extremum_channels_ids,'77'))

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([2]):
    wf = waveform_good.get_waveforms(unit_id)
    color = colors[i]
    ax.plot(wf[:, :,channel_association_dict[824]].T, color=color, lw=0.3)
print(wf.shape)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([104]):
    wf = waveform_good.get_waveforms(unit_id)
    ax.plot(wf[:, :,channel_association_dict[973]].T, color=colors[0], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[50]].T, color=colors[1], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[719]].T, color=colors[2], lw=0.3)
print(wf.shape)

In [ ]:
peak_shift=si.get_template_extremum_channel_peak_shift(waveform_good)

print(peak_shift)

In [ ]:
fig, ax = plt.subplots()
for i, unit_id in enumerate([52,53]):
    template = waveforms.get_template(unit_id)
    color = colors[i]
    ax.plot(template[:, channel_association_dict[907]].T, color=color, lw=3)
print(template.shape)

In [ ]:
#37 39

w = sw.plot_unit_templates(waveform_good, unit_ids=[22,29],plot_channels=False )

In [ ]:
w = sw.plot_unit_waveforms(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_templates(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_probe_map(waveform_good, unit_ids=[2,4,7])

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,5))
spike_times = {}
for unit_id in clean_sorting.get_unit_ids():
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=1*fs,end_frame=20*fs)
    if len(spike_train) > 0:
        spike_times[unit_id] = spike_train / float(fs)
        #print(spike_times[unit_id])
       # print(unit_id*np.ones_like(spike_times[unit_id]))
        ax1.plot(spike_times[unit_id],unit_id*np.ones_like(spike_times[unit_id]),
                             marker='|', mew=1, markersize=3,
                             ls='',color='black')
                             